In [1]:
import os
from config import OpenAPI_key
os.environ["OPENAI_API_KEY"] = OpenAPI_key
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain_openai import ChatOpenAI

## Data Generation for Real Estate Listings

In [2]:
from langchain_core.prompts import ChatPromptTemplate
model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name=model_name, temperature=0, max_tokens=4000)
# For LLM response in JSON format
json_llm = llm.bind(response_format={"type": "json_object"})
num_listings = 15
# Prompt for Synthetic Data Generation of Real Estate Listings
user_input = """
Generate a JSON list of {} diverse real estate listings in which each object has keys deduced from the example listing.

**Example Listing:**

Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""
user_input = user_input.format(num_listings)
listing_generation_template = ChatPromptTemplate.from_messages([
    ("system", "You are a real estate assistant specializing in creating detailed and attractive real estate listings. Each listing should include information about the neighborhood, price, number of bedrooms and bathrooms, house size, and descriptions of both the house and the neighborhood. Follow the format and style of the example listing provided below."),
    ("human", "{user_input}"),
])
listing_generation_prompt = listing_generation_template.invoke(
    {
        "user_input": user_input
    }
)
print(listing_generation_prompt)

ai_msg = json_llm.invoke(listing_generation_prompt)

messages=[SystemMessage(content='You are a real estate assistant specializing in creating detailed and attractive real estate listings. Each listing should include information about the neighborhood, price, number of bedrooms and bathrooms, house size, and descriptions of both the house and the neighborhood. Follow the format and style of the example listing provided below.'), HumanMessage(content='\nGenerate a JSON list of 15 diverse real estate listings in which each object has keys deduced from the example listing.\n\n**Example Listing:**\n\nNeighborhood: Green Oaks\nPrice: $800,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2,000 sqft\n\nDescription: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and di

### Separating Quantitative and Textual data from Generated Listings.

In [3]:
import json
## Storing generated data in JSON and txt format

# with open("Generated_listings.txt", "w") as text_file:
#     text_file.write(ai_msg.content)
# gen_listings_json = json.loads(ai_msg.content)
# print(type(gen_listings_json))
# with open('Generated_listings.json', 'w') as f:  
#     json.dump(gen_listings_json, f)

## Loading saved JSON data
f = open('listings.json')
data_listings = json.load(f)
print(type(data_listings), len(data_listings['listings']))

semantic_keys = ['description', 'neighborhood_description']
listing_texts = []
listing_metadata = []
for listing in data_listings['listings']:
    curr_text = ''
    for i, sk in enumerate(semantic_keys):
        curr_text += sk + ': '
        if i < len(semantic_keys)-1:
            curr_text += listing[sk] + ' '
        else:
            curr_text += listing[sk] # No space after adding the text of the last semantic key
    listing_texts.append(curr_text)
    dict_metadata = {k: listing[k] for k in listing.keys() if k not in semantic_keys}
    listing_metadata.append(dict_metadata)



<class 'dict'> 15


### Generating and Storing Embeddings in Vector Database

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
vector_db = Chroma.from_texts(listing_texts, embedding=embeddings, metadatas=listing_metadata, persist_directory="./chroma_db_listings")

### User Preference Interface

In [84]:
preference_questions = [   
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
## For interactive QA
# user_answers = [ ] 
# for question in preference_questions:
#    answer = input(question)
#    user_answers.append(answer)

## EXAMPLE 1
# user_answers = [
#     "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
#     "A quiet neighborhood, good local schools, and convenient shopping options.",
#     "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
#     "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
#     "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
# ]

## EXAMPLE 2
user_answers = [
    "A luxurious four-bedroom house with a modern kitchen and good ventilation in the living room.",
    "Proximity to sea, quiet neighborhood, and lots of natural light",
    "A backyard with a pool, a home office space, and modern appliances.",
    "Easy access to a major highways and gas stations.",
    "Suburban, with a mix of residential areas and local shops."
]

### Constructing VectorDB Query and Searching Based on Preferences

In [87]:
from langchain_community.chat_message_histories import ChatMessageHistory
# Generating an efficient search query based on the chat history of User Preferences
model_name = 'gpt-4o-mini'
llm = ChatOpenAI(model_name=model_name, temperature=0, max_tokens=1000)
pref_chat_history = ChatMessageHistory()
for i in range(len(preference_questions)):
    pref_chat_history.add_ai_message(preference_questions[i])
    pref_chat_history.add_user_message(user_answers[i])
user_input = "Given the following conversation between a home buyer and a real estate agent, construct an efficient text search query that can be used to fetch semantically similar real estate listings from a database. Just provide the search query in the response. Ensure the search query is in simple comma-separated format."
listing_search_template = ChatPromptTemplate.from_messages([
    ("human", "{user_input}"),
    *pref_chat_history.messages,
])
listing_search_prompt = listing_search_template.invoke(
    {
        "user_input": user_input
    }
)
# print(listing_search_prompt)

ai_msg = llm.invoke(listing_search_prompt)
search_query = ai_msg.content
print(search_query)

# Retrieving 3 most similar documents by utilizing the generated search query
similar_docs = vector_db.similarity_search(search_query, k=3)

luxurious, four-bedroom, modern kitchen, good ventilation, proximity to sea, quiet neighborhood, natural light, backyard, pool, home office, modern appliances, easy access, major highways, gas stations, suburban, residential areas, local shops


### Personalizing Listing Descriptions

In [88]:
def reformat_original_listing(in_content, k1 = 'description:', k2 = 'neighborhood_description:'):
    '''
    Reformat Original Listing to match the format as per the Augmented Listing.
    '''
    out_content = "**Description:** {} \n\n **Neighborhood Description:** {}"
    desc_x = in_content[in_content.index(k1)+len(k1):in_content.index(k2)]
    ndesc_x = in_content[in_content.index(k2)+len(k2):]
    out_content = out_content.format(desc_x, ndesc_x)
    return out_content

search_augmentation_prompt = """
You are an expert and honest real estate agent. Your task is to rephrase the given real estate listing to subtly emphasize the aspects that align with the provided user preferences. Make sure not to alter any factual information or add any extra details.

***IMPORTANT INSTRUCTIONS***: 
1. Keep the description accurate and factual as per the original listing.
2. Do **NOT** add any new information that is missing in the original listing.

### User Preferences: 
{}

### Original Real Estate Listing:
{}

***Remember to follow these instructions:***: 
1. Keep the description accurate and factual as per the original listing.
2. Do **NOT** add any new information that is missing in the original listing.
"""
model_name = 'gpt-4o-mini'
llm = ChatOpenAI(model_name=model_name, temperature=0, max_tokens=1000)
print('User Preferences: ')
print(search_query)
for doc in similar_docs:
    curr_search_augmentation_prompt = search_augmentation_prompt.format(search_query, doc.page_content)
    # print(curr_search_augmentation_prompt)
    print('Original Real Estate Listing: ')
    print(reformat_original_listing(doc.page_content))
    ai_msg = llm.invoke(curr_search_augmentation_prompt)
    print('Augmented Real Estate Listing: ')
    print(ai_msg.content, '\n\n')

    

User Preferences: 
luxurious, four-bedroom, modern kitchen, good ventilation, proximity to sea, quiet neighborhood, natural light, backyard, pool, home office, modern appliances, easy access, major highways, gas stations, suburban, residential areas, local shops
Original Real Estate Listing: 
**Description:**  Escape to this stunning 3-bedroom, 2.5-bathroom retreat by the sea. The house features a gourmet kitchen, a sun-drenched living room, and a private deck overlooking the ocean. Enjoy the sound of waves crashing and salty sea breezes in this coastal paradise.  

 **Neighborhood Description:**  Seaside Retreat offers a peaceful coastal lifestyle with sandy beaches, seaside cafes, and water activities. Residents can watch breathtaking sunsets, go surfing or paddleboarding, and explore tide pools along the shore. Experience the magic of seaside living in this idyllic retreat.
Augmented Real Estate Listing: 
**Description:** Discover this exquisite 3-bedroom, 2.5-bathroom retreat by th

#### Example 1
**User Preferences:** 
"three-bedroom house, spacious kitchen, cozy living room, quiet neighborhood, good local schools, convenient shopping, backyard, two-car garage, modern heating system, easy access to bus line, proximity to highway, bike-friendly roads, suburban tranquility, urban amenities"

1. Original Real Estate Listing:

    **Description:**  Escape to this tranquil 3-bedroom, 2-bathroom home nestled in the peaceful Tranquil Pines neighborhood. The house features a cozy fireplace, a sunlit breakfast nook, and a private backyard surrounded by pine trees. Enjoy the serenity of nature and the chirping of birds in this wooded retreat.  

    **Neighborhood Description:**  Tranquil Pines offers a serene escape from the hustle and bustle of city life with winding nature trails and birdwatching spots. Residents can explore nearby forests, go camping, or simply relax in the shade of towering pines. Experience the peace and quiet of woodland living in this idyllic neighborhood.

    Augmented Real Estate Listing: 

    **Description:** Discover this inviting 3-bedroom, 2-bathroom home located in the serene Tranquil Pines neighborhood. This residence boasts a cozy living room with a charming fireplace, a spacious kitchen with a sunlit breakfast nook, and a private backyard surrounded by lush pine trees, perfect for outdoor relaxation. Embrace the peaceful ambiance of nature and the soothing sounds of birds in this delightful retreat.

    **Neighborhood Description:** Tranquil Pines provides a peaceful escape from the hustle and bustle of city life, featuring winding nature trails and birdwatching spots. Residents can enjoy nearby forests, go camping, or simply unwind in the shade of the towering pines. Experience the suburban tranquility of this idyllic neighborhood while still being close to urban amenities. 


2. Original Real Estate Listing: 

    **Description:**  Step into this spacious 4-bedroom, 3-bathroom home in the serene Willow Creek neighborhood. The house features a modern kitchen with granite countertops, a cozy fireplace in the living room, and a large backyard perfect for entertaining. Enjoy the peaceful surroundings and easy access to hiking trails and parks.  

    **Neighborhood Description:**  Willow Creek is a family-friendly neighborhood known for its top-rated schools and lush greenery. Residents can enjoy community events at the local park and have convenient access to shopping centers and restaurants.

    Augmented Real Estate Listing: 

    **Description:** Discover this inviting 4-bedroom, 3-bathroom home nestled in the tranquil Willow Creek neighborhood. The residence boasts a modern kitchen with granite countertops, a cozy fireplace in the living room, and a spacious backyard ideal for gatherings. Experience the peaceful ambiance and enjoy easy access to nearby hiking trails and parks.

    **Neighborhood Description:** Willow Creek is a family-friendly community celebrated for its top-rated schools and lush greenery. Residents can partake in community events at the local park and benefit from convenient access to shopping centers and restaurants, all while enjoying the suburban tranquility that this area offers. 


3. Original Real Estate Listing: 

    **Description:**  Live in style in this 3-bedroom, 2.5-bathroom home perched in the desirable Sunnyvale Heights neighborhood. The house features a modern kitchen, a spacious master suite, and a backyard oasis with a pool and spa. Enjoy the sunny California weather and panoramic views from this hilltop retreat.  

    **Neighborhood Description:**  Sunnyvale Heights offers panoramic views of the city skyline and easy access to hiking trails and parks. Residents can enjoy outdoor activities, visit nearby vineyards, and dine at local eateries with stunning views. Experience the best of California living in this sunny hillside neighborhood.
    
    Augmented Real Estate Listing: 
    
    **Description:** Experience comfortable living in this charming 3-bedroom, 2.5-bathroom home located in the peaceful Sunnyvale Heights neighborhood. The residence boasts a modern kitchen, perfect for culinary enthusiasts, and a cozy living room ideal for relaxation. Step outside to your own backyard oasis, complete with a pool and spa, where you can unwind and enjoy the sunny California weather. 

    **Neighborhood Description:** Sunnyvale Heights not only offers serene suburban tranquility but also provides panoramic views of the city skyline. With easy access to hiking trails and parks, residents can engage in outdoor activities while being close to good local schools and convenient shopping options. Enjoy the best of both worlds with urban amenities just a short distance away, all within this bike-friendly community. 




#### Example 2

**User Preferences:** 
luxurious, four-bedroom, modern kitchen, good ventilation, proximity to sea, quiet neighborhood, natural light, backyard, pool, home office, modern appliances, easy access, major highways, gas stations, suburban, residential areas, local shops

1. Original Real Estate Listing: 

    **Description:**  Escape to this stunning 3-bedroom, 2.5-bathroom retreat by the sea. The house features a gourmet kitchen, a sun-drenched living room, and a private deck overlooking the ocean. Enjoy the sound of waves crashing and salty sea breezes in this coastal paradise.  

    **Neighborhood Description:**  Seaside Retreat offers a peaceful coastal lifestyle with sandy beaches, seaside cafes, and water activities. Residents can watch breathtaking sunsets, go surfing or paddleboarding, and explore tide pools along the shore. Experience the magic of seaside living in this idyllic retreat.

    Augmented Real Estate Listing: 

    **Description:** Discover this exquisite 3-bedroom, 2.5-bathroom retreat by the sea, perfect for those seeking a luxurious coastal lifestyle. The home boasts a gourmet kitchen equipped with modern appliances, a sun-drenched living room that invites natural light, and a private deck that offers stunning views of the ocean. Enjoy the soothing sound of waves and refreshing sea breezes in this tranquil paradise.

    **Neighborhood Description:** Seaside Retreat provides a serene coastal environment, ideal for relaxation and leisure. With sandy beaches, charming seaside cafes, and a variety of water activities, residents can indulge in breathtaking sunsets, surfing, paddleboarding, and exploring tide pools. Experience the allure of seaside living in this peaceful and inviting community. 


2. Original Real Estate Listing: 

    **Description:**  Welcome to this luxurious 4-bedroom, 3.5-bathroom home with breathtaking views of the harbor. The house features a chef's kitchen, a wine cellar, a home office, and a spacious deck for watching sailboats glide by. Live the waterfront lifestyle in this elegant coastal retreat.  

    **Neighborhood Description:**  Harbor View offers exclusive waterfront living with access to marinas, yacht clubs, and seaside dining. Residents can sail, fish, or simply relax by the water's edge while enjoying stunning sunsets over the harbor. Experience the beauty and tranquility of coastal living in this prestigious neighborhood.
    
    Augmented Real Estate Listing: 

    **Description:** Welcome to this luxurious 4-bedroom, 3.5-bathroom home that boasts breathtaking views of the harbor. This residence features a modern chef's kitchen equipped with high-end appliances, a dedicated home office, and a spacious deck perfect for enjoying the serene surroundings and watching sailboats glide by. Embrace the waterfront lifestyle in this elegant coastal retreat, designed for good ventilation and abundant natural light.

    **Neighborhood Description:** Harbor View offers exclusive waterfront living with easy access to marinas, yacht clubs, and seaside dining. Residents can enjoy a quiet neighborhood while engaging in activities like sailing and fishing, or simply relaxing by the water's edge, all while taking in stunning sunsets over the harbor. Experience the beauty and tranquility of coastal living in this prestigious suburban area, conveniently located near major highways and local shops. 


3. Original Real Estate Listing: 

    **Description:**  Welcome to this luxurious 5-bedroom, 4-bathroom estate overlooking the serene Lakefront. The house boasts a gourmet kitchen, a home theater, a private dock, and a spacious deck for enjoying the sunset views. Perfect for entertaining guests or relaxing in the lap of luxury.  

    **Neighborhood Description:**  Lakefront Estates offers exclusive waterfront living with access to boating, fishing, and water sports. Residents can unwind in the peaceful surroundings or host gatherings in the community clubhouse. Enjoy the best of lakeside living in this prestigious neighborhood.

    Augmented Real Estate Listing: 
    
    **Description:** Welcome to this luxurious 5-bedroom, 4-bathroom estate overlooking the serene Lakefront. This home features a gourmet kitchen equipped with modern appliances, perfect for culinary enthusiasts. The spacious layout allows for good ventilation and abundant natural light throughout. Enjoy the private dock and spacious deck, ideal for entertaining guests or relaxing while taking in the stunning sunset views. 

    **Neighborhood Description:** Lakefront Estates offers exclusive waterfront living with easy access to boating, fishing, and water sports. This quiet neighborhood provides a peaceful retreat, allowing residents to unwind in tranquil surroundings. The community clubhouse is perfect for hosting gatherings, making it an ideal setting for both relaxation and socializing. Experience the best of lakeside living in this prestigious suburban area. 


